In [38]:
import requests
import os
import re
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
import time
import matplotlib.pyplot as plt
import urllib
%matplotlib inline
import pickle
from tqdm import tqdm


In [29]:
manga = 'https://myanimelist.net/topmanga.php'

headers = {
    'Accept-Language': 'en',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'X-FORWARDED-FOR': '2.21.184.0'
}

response = requests.get(manga, headers=headers)
print(response.status_code)

200


In [30]:
mangasoup = BeautifulSoup(response.text, "lxml")

In [31]:
# mangasoup.find_all('a')
# mangasoup.find_all('h3', attrs={'class': "manga_h3"})


In [32]:
mangasoup.find_all('a', attrs={'class': 'next'})[0]['href']

'?limit=50'

In [33]:
  

def get_links_to_movies(soup):
    h3s = soup.find_all('h3', attrs={'class': "manga_h3"})
    movie_links = {el.find('a').text: el.find('a')['href'] for el in h3s}
    return movie_links    

In [34]:
def get_link_to_next_page(soup):
    links = soup.find_all('a', attrs={'class': 'next'})
    if len(links) == 0:
        return None
    else:
        link = links[0]['href']
        return link

In [35]:
# def get_all_links(url):
#     headers = {'Accept-Language': 'en',
#                'X-FORWARDED-FOR': '2.21.184.0'}
    
#     all_movies_links = {}
    
#     while True:
#         response = requests.get(url, headers=headers)
#         assert response.status_code == 200
#         soup = BeautifulSoup(response.text)
#         movie_links = get_links_to_movies(soup)
#         movie_links = {movie: urllib.parse.urljoin(url, movie_link) for (movie, movie_link) in movie_links.items()}
        
#         all_movies_links.update(movie_links)
#         print(f'number of links so far: { len(all_movies_links)}')
#         link = get_link_to_next_page(soup)
#         if not link:
#             return all_movies_links
#         url = urllib.parse.urljoin(url, link)
#         time.sleep(1)

In [39]:

def get_all_links(url, max_links=30000):
    headers = {
        'Accept-Language': 'en',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'X-FORWARDED-FOR': '2.21.184.0'
    }
    
    all_movies_links = {}
    
    # Initialize tqdm progress bar
    pbar = tqdm(total=max_links, desc="Collecting movie links")
    
    while True:
        response = requests.get(url, headers=headers)
        assert response.status_code == 200
        soup = BeautifulSoup(response.text, 'html.parser')
        movie_links = get_links_to_movies(soup)
        movie_links = {movie: urllib.parse.urljoin(url, movie_link) for (movie, movie_link) in movie_links.items()}
        
        new_links_count = len(movie_links)
        all_movies_links.update(movie_links)
        
        # Update progress bar
        pbar.update(min(new_links_count, max_links - pbar.n))
        
        if len(all_movies_links) >= max_links:
            pbar.close()
            print(f'Reached {max_links} links. Stopping.')
            return dict(list(all_movies_links.items())[:max_links])
        
        link = get_link_to_next_page(soup)
        if not link:
            pbar.close()
            return all_movies_links
        url = urllib.parse.urljoin(url, link)
        time.sleep(1)


In [40]:

all_manga_links = get_all_links(manga)

Reached 30000 links. Stopping.


In [42]:
with open('all_manga_links', 'wb') as f:
    pickle.dump(all_manga_links, f)